In [1530]:
import pandas as pd;
import numpy as np;
import seaborn as sns;
import matplotlib.pyplot as plt;
from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)

data has fill all nans

In [1533]:
dataset = pd.read_feather('dataset.csv')
y = pd.read_feather('y.csv')
ids = pd.read_feather('Id.csv')

In [1534]:
sum (dataset.isnull().sum())

0

In [1535]:
dataset.shape

(2919, 81)

In [1536]:
y.shape

(1460, 1)

In [1537]:
ids.shape

(1459, 1)

In [1538]:
cat_dataset = dataset.select_dtypes(include=['object','bool'])
for c in cat_dataset : cat_dataset[c] = cat_dataset[c].astype('category')
cat_dataset_bk = cat_dataset 


/opt/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [1539]:
# turn string cat to numeric string cat
for c in cat_dataset : cat_dataset[c] = cat_dataset[c].cat.codes + 1

/opt/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [1540]:
num_dataset = dataset.select_dtypes(exclude=['object','bool']).astype('float32')

In [1541]:
np.where(num_dataset < 0)

(array([1876, 2549]), array([36, 37]))

In [1542]:
num_dataset.loc[ num_dataset['RemodAfterBuilt'] < 0,'RemodAfterBuilt'] = 0
num_dataset.loc[ num_dataset['SoldAfterBuilt'] < 0,'SoldAfterBuilt'] = 0

In [1543]:
np.where(num_dataset < 0) 

(array([], dtype=int64), array([], dtype=int64))

unskew data

In [1544]:
from scipy.stats import skew
skew_feats = num_dataset.apply(skew).sort_values(ascending=False)
skew_feats.head(10)

YearGaBuiltNow    28.399906
MiscVal           21.947193
PoolArea          16.898327
LotArea           12.822433
LowQualFinSF      12.088761
3SsnPorch         11.376063
KitchenAbvGr       4.302255
BsmtFinSF2         4.146142
EnclosedPorch      4.003891
ScreenPorch        3.946693
dtype: float64

In [1545]:
skew_feats = skew_feats[abs(skew_feats) > 1]

for feat in skew_feats.index:
    num_dataset[feat] = np.log1p(num_dataset[feat])

min-max or standard normalazation num_features

In [1546]:
for x in num_dataset:
     num_dataset[x] = (num_dataset[x] - num_dataset[x].mean()) / (num_dataset[x].std())
#         num_dataset[x] = (num_dataset[x] - num_dataset[x].mean()) / (num_dataset[x].max() - num_dataset[x].min())

In [1547]:
cat_dataset = cat_dataset.astype('float32')

In [1548]:
dataset = pd.concat([cat_dataset,num_dataset],axis=1)
dataset.to_feather('dataset_after_unskew.csv')

In [1549]:
dataset = pd.read_feather('dataset_after_unskew.csv')
X = dataset[:train_len]
X_test   = dataset[train_len:]
sum(X.isnull().sum())

0

In [1550]:
sum (np.isnan(num_dataset).any())

0

In [1551]:
np.where(np.isnan(num_dataset))

(array([], dtype=int64), array([], dtype=int64))

In [1552]:
sum(X_test.isnull().sum() )
X.shape

(1460, 81)

In [1553]:
X_test.shape

(1459, 81)

### Prepare everything for fastai

In [1554]:
cat_dataset.dtypes.unique()

array([dtype('float32')], dtype=object)

In [1555]:
dataset.shape

(2919, 81)

In [1556]:
from sklearn.model_selection import train_test_split,StratifiedKFold

In [1557]:
indices = range(len(X))

get train ,cv and indices

In [1558]:
X_train,X_cv,y_train,y_cv,indices_train,indices_cv= train_test_split(X,y,indices,test_size=0.2,shuffle=True)
val_idx = indices_train

In [1559]:
yl = np.log(y)
yl = yl['SalePrice']

In [1560]:
max_log_y = np.max(yl)
y_range = (0, max_log_y*1.2)

def inv_y(a): return np.exp(a)


def exp_rmspe(y_pred, targ):
    targ = inv_y(targ)
    pct_var = (targ - inv_y(y_pred))/targ
    return math.sqrt((pct_var**2).mean())

In [1561]:
def rmse(y_pred, targ):
    pct_var = (targ - (y_pred))
    return math.sqrt( torch.sum( (pct_var**2) ) / len(y_pred) )

In [1562]:
cat_vars = cat_dataset.columns.tolist();len(cat_vars)

43

### Build Models

In [1563]:
from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)

PATH='../data/rossmann/'
warnings.filterwarnings(action='ignore', category=UserWarning)

In [1564]:
X_test_bk = X_test.copy()
X_test.reset_index(drop=True);
X_test.shape

(1459, 81)

In [1565]:
len(val_idx)

1168

In [1566]:
X.shape

(1460, 81)

In [1567]:
1460*0.2

292.0

In [1568]:
md = ColumnarModelData.from_data_frame(PATH, val_idx, X, yl.astype('float32'), cat_flds=cat_vars, bs=128,
                                       test_df=X_test)

In [1574]:
cat_dataset_bk = cat_dataset_bk.astype('category')

In [1610]:
cat_sz = [(c, len(cat_dataset_bk[c].cat.categories)) for c in cat_dataset_bk] ; 
cat_sz;

In [1576]:
# emb_szs = [(c, (c+1)//2) for _,c in cat_sz] ;emb_szs
emb_szs = [(c, c) for _,c in cat_sz] ;emb_szs;

In [1577]:
X.columns;

In [1578]:
 len(X.columns)

81

In [1579]:
len(cat_vars)

43

In [1580]:
cat_vars

['Alley',
 'BldgType',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'BsmtQual',
 'CentralAir',
 'Condition1',
 'Condition2',
 'Electrical',
 'ExterCond',
 'ExterQual',
 'Exterior1st',
 'Exterior2nd',
 'Fence',
 'FireplaceQu',
 'Foundation',
 'Functional',
 'GarageCond',
 'GarageFinish',
 'GarageQual',
 'GarageType',
 'Heating',
 'HeatingQC',
 'HouseStyle',
 'KitchenQual',
 'LandContour',
 'LandSlope',
 'LotConfig',
 'LotShape',
 'MSZoning',
 'MasVnrType',
 'MiscFeature',
 'Neighborhood',
 'PavedDrive',
 'RoofMatl',
 'RoofStyle',
 'SaleCondition',
 'SaleType',
 'Street',
 'Utilities',
 'FullnullBsmt']

In [1581]:
len(emb_szs)

43

In [1652]:
#score 0.16 embedding for full without reducing feature unique nums 
# m = md.get_learner(emb_szs, len(X.columns)-len(cat_vars) , 0.08,  1, [22,10], [0.05,0.05], y_range=y_range)
m = md.get_learner(emb_szs, len(X.columns)-len(cat_vars) , 0.08,  1, [59,10], [0.06,0.09], y_range=y_range)

# m = md.get_learner(emb_szs, len(X.columns)-len(cat_vars) , 0.1,  1, [40,20], [0.06,0.05], y_range=y_range)

In [1714]:
m.fit(0.0001,5, metrics=[rmse],cycle_len=2,cycle_mult=1)

epoch      trn_loss   val_loss   rmse              
    0      0.008183   0.024417   0.155235  
    1      0.007832   0.024402   0.155174          
    2      0.006691   0.024353   0.154972          
    3      0.006297   0.024345   0.154941          
    4      0.006121   0.024335   0.154906          
    5      0.005839   0.024335   0.154906          
    6      0.00643    0.024317   0.154832          
    7      0.006568   0.02431    0.15481           
    8      0.006444   0.024287   0.154734          
    9      0.006453   0.024283   0.154726          


[array([0.02428]), 0.1547257846453938]

In [1715]:
pred = m.predict(is_test=True)

In [1716]:
pred = np.expm1(pred)
pred.shape
pred = pred.flatten()
pred.shape

(1459,)

In [1717]:
test = pd.read_csv('../data/house-prices/test.csv')
ids = test['Id']
ids.shape

(1459,)

In [1718]:
result  = pd.concat([ids,pd.Series(pred).astype('float64')],axis=1);
result.columns = ['Id','SalePrice']
result.to_csv('fastai-model.csv',index=False)